In [3]:
import argparse
import os
import re
import numpy as np
import time as t
from multiprocessing import Pool, cpu_count
from functools import partial
import os

In [4]:
input_dir = '/home/muskaanjain/ntu/Muskaan/graph_2D_CNN/datasets/ten/train/'
path_to_embeddings = '/home/muskaanjain/ntu/Muskaan/graph_2D_CNN/datasets/ten/'
path_to_hist = '/home/muskaanjain/ntu/Muskaan/graph_2D_CNN/datasets/hist/'
dataset = 'train'
definition = 14
max_n_channels = 5

In [5]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]


n_dim = 2*max_n_channels
    
all_file_names  = os.listdir(path_to_embeddings + dataset + '/') 
print('===== total number of files in folder: =====', len(all_file_names))

file_names_filtered = all_file_names
file_names_filtered.sort(key=natural_keys)
print('===== number of files after filtering: =====', len(file_names_filtered))
print('*** head ***')
print(file_names_filtered[:5])
print('*** tail ***')
print(file_names_filtered[-5:])
    
# load tensors
tensors = []
for idx, name in enumerate(file_names_filtered):
    tensor = np.load(path_to_embeddings + dataset + '/' + name)
    tensors.append(tensor)
    #print(tensor.shape)

tensors = np.array(tensors)
full = []

for l in tensors:
    s = l.ravel()    
    full += s.tolist()

print('tensors loaded')
full = np.array(full)
my_max = np.amax(full)
my_min = np.amin(full)
print('range:', my_max, my_min)

===== total number of files in folder: ===== 210
===== number of files after filtering: ===== 210
*** head ***
['0.npy', '1.npy', '2.npy', '3.npy', '4.npy']
*** tail ***
['205.npy', '206.npy', '207.npy', '208.npy', '209.npy']
tensors loaded
range: 4.833520889282227 -4.588778495788574


In [6]:
full.shape

(25255936,)

In [14]:
def get_subgraph_ids(): 
    subgraphs_in_graphs = []
    graph_files  = os.listdir('/home/muskaanjain/ntu/Muskaan/gspannew/id/train_np/') 
    for idx, name in enumerate(graph_files):
        graph = np.load('/home/muskaanjain/ntu/Muskaan/gspannew/id/train_np/g_' + str(idx) + '.npy')
        #print(str(idx) + '.npy')
        subgraphs_in_graphs.append(graph)

    return subgraphs_in_graphs
        
def find_nearest_smaller(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    if(array[idx] <= value):
        return idx
    else:
        return idx-1

def get_hist_node2vec(emb,graph_id,d,my_min,my_max,definition):
    # d should be an even integer
    subgraphs_in_graphs = get_subgraph_ids()
    img_dim = int(np.arange(my_min, my_max+0.05,(my_max+0.05-my_min)/float(definition*(my_max+0.05-my_min))).shape[0]-1)
    #print(img_dim)
    my_bins = np.linspace(my_min,my_max,img_dim) #  to have middle bin centered on zero
    Hs = []
    Hind = []
    print('number of embeddings: ', len(emb))
    print(len(subgraphs_in_graphs[graph_id]))
    for i in range(0,d,2):
        H, xedges, yedges = np.histogram2d(x=emb[:,i],y=emb[:,i+1],bins=my_bins, normed=False)
        #h = np.zeros(H.shape)
        h = {}
        #can have better way
       
        for l in range(len(H)):
            for m in range(H.shape[1]):
                address = m
                if(l != 0):
                    address += (l*H.shape[1])
                    
                h[address] = []
                
        for j in range(len(emb)):
            idx = find_nearest_smaller(xedges, emb[j, i])
            idy = find_nearest_smaller(yedges, emb[j, i+1])
            address = idy
            if(idx != 0):
                address += (idx*H.shape[1])
                    
            #if graph_id == 3:
                #print(len(subgraphs_in_graphs[graph_id]))
            h[address].append(subgraphs_in_graphs[graph_id][j] + 1) #to regain the ids, subtract - 1 from the h[idx][idy]
            #print("h: ", h[idx][idy], " H: ", H[idx][idy])
            #print(np.sum(H))
        #print(np.array(H).shape)
        Hs.append(H)
        Hind.append(h)
        
        
    Hs = np.array(Hs)
    #print(len(Hind))
    Hind = np.array(Hind)
    return  Hs, Hind

def to_parallelize(dataset,n_dim,my_min,my_max,my_def,path_read,path_write):
    path_write_dataset = path_write + dataset + '/embed_hist/'
    l=0 
    for filename in os.listdir(input_dir):
    	l = l+1
    print(l)
    i=0
        
    while i < l:
        emb = np.load(path_read +str(i)+".npy")
        #print(emb.shape)
        if emb.shape[0] == 1024:
            emb = emb.reshape((1,emb.shape[0]))
            emb = emb[0,:n_dim]
            emb = emb.reshape((1,emb.shape[0]))
            #print(emb.shape)
            my_hist, hist_with_ind = get_hist_node2vec(emb=emb,graph_id=i,d=n_dim,my_min=my_min,my_max=my_max,definition=definition)
            np.save(path_write_dataset + str(i), my_hist, allow_pickle=False)
            np.save('/home/muskaanjain/ntu/version1/graphimg_with_ids/train/' + str(i), hist_with_ind, allow_pickle=True)
            print(i, " saved")
            i=i+1
            
        elif emb.shape[0] != 0:
            emb = emb[:,:n_dim]
            #print(emb.shape)
            my_hist, hist_with_ind = get_hist_node2vec(emb=emb,graph_id=i,d=n_dim,my_min=my_min,my_max=my_max,definition=definition) 
            np.save(path_write_dataset + str(i), my_hist, allow_pickle=False)
            np.save('/home/muskaanjain/ntu/version1/graphimg_with_ids/train/' + str(i), hist_with_ind, allow_pickle=True)
            print(i, " saved")
            i = i+1

        else:
            img_dim = int(np.arange(my_min, my_max+0.05,(my_max+0.05-my_min)/float(definition*(my_max+0.05-my_min))).shape[0]-1)
            my_bins = np.linspace(my_min,my_max,img_dim)
            Hs = []
            Hind = []
            for _ in range(0,n_dim,2):
                H, xedges, yedges = np.histogram2d(x=[],y=[],bins=my_bins, normed=False)
                h = {}       
                for k in range(len(H)):
                    for m in range(H.shape[1]):
                        address = m
                        if(k != 0):
                            address += (k*H.shape[1])
                        h[address] = []
                        
                Hs.append(H)
                Hind.append(h)
        
            Hs = np.array(Hs)
            Hind = np.array(Hind)
            np.save(path_write_dataset + str(i), Hs, allow_pickle=False)
            np.save('/home/muskaanjain/ntu/version1/graphimg_with_ids/train/' + str(i), Hind, allow_pickle=True)
            print(i, " saved")
            i=i+1

to_parallelize(dataset=dataset, n_dim=n_dim, my_min=my_min, my_max=my_max, my_def=definition, path_read=path_to_embeddings + dataset + "/", path_write = path_to_hist)

210
number of embeddings:  1
1
0  saved
number of embeddings:  6
6
1  saved
number of embeddings:  99
99
2  saved
number of embeddings:  2
2
3  saved
number of embeddings:  6
6
4  saved
number of embeddings:  18
18
5  saved
number of embeddings:  255
255
6  saved
number of embeddings:  38
38
7  saved
number of embeddings:  71
71
8  saved
number of embeddings:  46
46
9  saved
number of embeddings:  242
242
10  saved
number of embeddings:  2
2
11  saved
number of embeddings:  165
165
12  saved
number of embeddings:  335
335
13  saved
number of embeddings:  55
55
14  saved
number of embeddings:  129
129
15  saved
number of embeddings:  34
34
16  saved
number of embeddings:  76
76
17  saved
number of embeddings:  8
8
18  saved
number of embeddings:  272
272
19  saved
number of embeddings:  280
280
20  saved
number of embeddings:  45
45
21  saved
number of embeddings:  25
25
22  saved
number of embeddings:  101
101
23  saved
number of embeddings:  34
34
24  saved
number of embeddings:  91
9

204  saved
number of embeddings:  285
285
205  saved
number of embeddings:  82
82
206  saved
number of embeddings:  144
144
207  saved
number of embeddings:  136
136
208  saved
number of embeddings:  73
73
209  saved
